In [3]:
# module set
import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common import exceptions

import time
import lxml

from collections import OrderedDict
from itertools import repeat

import matplotlib.pyplot as plt
import seaborn as sns

In [22]:
#피파인벤 연결
driver = webdriver.Chrome('./chromedriver')
player_url = 'http://fifaonline4.inven.co.kr/dataninfo/player/?code='

In [20]:
seasons = [['LH','TKL','20KL','19S', '19UCL', '20TOTY','20TOTY-N'],
          ['NG', 'COC', 'OTW', '19A', '19', 'HOT', '19TOTS'],
          ['ICON', 'TC', '18S', 'MC-ICON', '19TOTY', 'GR', '19KFA'],
          ['18A', 'TT', 'TB', 'MCFC', '18KFA', 'NHD', 'TKI'],
          ['18TOTY', '18', '17']]

In [6]:
def click_by_class(x, y):
    driver.get('http://fifaonline4.inven.co.kr/dataninfo/player/')
    driver.implicitly_wait(5)
#     //*[@id="fifaonline4Db"]/div[2]/ul[1]/li[2]/div[2]/div[2]/label[1]/span
    temp_path = f'//*[@id="fifaonline4Db"]/div[2]/ul[1]/li[2]/div[2]/div[{x}]/label[{y}]/span'
    button = driver.find_element_by_xpath(temp_path).click()
    driver.implicitly_wait(10)
    search = driver.find_element_by_xpath('//*[@id="fifaonline4Db"]/div[2]/ul[2]/li[17]/button[1]/img')
    search.click()
    driver.implicitly_wait(10)

In [7]:
def players_info(html):
    soup = bs(html, 'html.parser')
    search_url_list = []
    
    for link in soup.find_all('a'):
        if 'href' in link.attrs:
            li = link.attrs['href'].split('\n')
            if li[0].startswith(player_url):
                search_url_list.append(li[0])
            else:
                continue
    names = [name.text for name in soup.select('span > b')]
    search_url_list = list(OrderedDict(zip(search_url_list, repeat(None))))
    print("num of url : ", len(search_url_list))
    print(search_url_list[:5])
    return search_url_list, names

In [15]:
def get_comment(urls, names):
    total_df = pd.DataFrame()
    for i in range(len(urls)):
        driver.get(urls[i])
        driver.implicitly_wait(2)

        html = driver.page_source
        one_soup = bs(html, 'html.parser')
        one_player_db=one_soup.find_all('span', class_='comment')
        #one_player_name = one_soup.find('div', class_="fifa4 tooltip_title")
        #player_name = one_player_name.text.strip()
        player_list = [one_player_db]
        try:
            while True: # 댓글 페이지가 몇개인지 모르므로.            
                element = driver.find_element_by_xpath('//*[@id="cmtPaging"]/span/span[3]/a')
                driver.execute_script("arguments[0].click();", element)
                driver.implicitly_wait(2)
                html = driver.page_source
                one_soup = bs(html, 'html.parser')
                comments = one_soup.find_all('span', class_='comment')
                player_list.append(comments)

        except exceptions.ElementNotVisibleException as e: # 페이지 끝
            pass

        except Exception as e: # 다른 예외 발생시 확인
            print(e)

        cmt_list = []
        for db in player_list:
            for cmt in db:
                cmt_list.append(cmt.text)
        one_df = pd.DataFrame({'name': names[i], 'comment': cmt_list})
        total_df = pd.concat([total_df, one_df], axis = 0)
    total_df = total_df.reset_index(drop=True)
    return total_df

In [ ]:
for i in range(0, 4):
    for j in range(7):
        click_by_class(i+1, j+1)
        time.sleep(10)
        html = driver.page_source
        urls, names = players_info(html)
        comment_df = get_comment(urls, names)
        comment_df.to_csv("./comment_data/"+seasons[i][j]+'_comment.csv', mode='w')
   


In [15]:
def get_comment_ex(urls):
    total_df = pd.DataFrame()
    for i in range(2):
        driver.get(urls[i])
        driver.implicitly_wait(2)

        html = driver.page_source
        one_soup = bs(html, 'html.parser')
        one_player_db=one_soup.find_all('span', class_='comment')
        one_player_name = one_soup.find('div', class_="fifa4 tooltip_title")
        player_name = one_player_name.text.strip()
        player_list = [one_player_db]
        try:
            while True: # 댓글 페이지가 몇개인지 모르므로.            
                element = driver.find_element_by_xpath('//*[@id="cmtPaging"]/span/span[3]/a')
                driver.execute_script("arguments[0].click();", element)
                driver.implicitly_wait(2)
                html = driver.page_source
                one_soup = bs(html, 'html.parser')
                comments = one_soup.find_all('span', class_='comment')
                player_list.append(comments)

        except exceptions.ElementNotVisibleException as e: # 페이지 끝
            pass

        except Exception as e: # 다른 예외 발생시 확인
            print(e)

        cmt_list = []
        for db in player_list:
            for cmt in db:
                cmt_list.append(cmt.text)
        one_df = pd.DataFrame({'name': player_name, 'comment': cmt_list})
        total_df = pd.concat([total_df, one_df], axis = 0)
    total_df = total_df.reset_index(drop=True)
    return total_df

In [23]:
#apply each class
click_by_class(3, 5)
time.sleep(10)
html = driver.page_source
urls, names = players_info(html)
comment_df = get_comment(urls, names)
comment_df.to_csv("./comment_data/"+seasons[2][4]+'_comment.csv', mode='w')

num of url :  12
['http://fifaonline4.inven.co.kr/dataninfo/player/?code=211020801', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=211158023', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=211177003', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=211155862', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=211192985']
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message

In [13]:
#check saved data
for i in range(3,4):
    for j in range(6):
        data = pd.read_csv('./comment_data/'+seasons[i][j]+"_comment.csv")
        data.head()
        click_by_class(i+1, j+1)
        time.sleep(10)
        html = driver.page_source
        urls, names = players_info(html)
        original = data['name'].unique().tolist()
        name_dict = {}
        for origin, new in zip(original, names):
            name_dict[origin] = new
        data['name'] = data['name'].apply(lambda x: name_dict[x])
        data.head()

        data.to_csv("./comment_data/"+seasons[i][j]+'_comment.csv', mode='w')

num of url :  346
['http://fifaonline4.inven.co.kr/dataninfo/player/?code=500158023', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=500020801', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=500183277', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=500190871', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=500209331']
num of url :  321
['http://fifaonline4.inven.co.kr/dataninfo/player/?code=207190042', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=207190043', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=207020801', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=207037576', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=207190871']
num of url :  500
['http://fifaonline4.inven.co.kr/dataninfo/player/?code=206020801', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=206158023', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=206182521', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=206183277', 'http

In [17]:
for j in range(3):
    click_by_class(5, j+1)
    time.sleep(10)
    html = driver.page_source
    urls, names = players_info(html)
    comment_df = get_comment(urls, names)
    comment_df.to_csv("./comment_data/"+seasons[4][j]+'_comment.csv', mode='w')


num of url :  11
['http://fifaonline4.inven.co.kr/dataninfo/player/?code=203020801', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=203158023', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=203155862', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=203177003', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=203192985']
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to loca

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to loca

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to loca

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to loca

...

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to loca

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to loca

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to loca

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to loca

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmtPaging"]/span/span[3]/a"}
  (Session info: chrome=79.0.3945.117)

Message: no such element: Unable to loca

In [19]:
data = pd.read_csv('./comment_data/'+seasons[4][2]+"_comment.csv")
data

,Unnamed: 0,name,comment
0,0,이니에스타,이니에스타 9카 팔아요 적당한 알파에 팔아드립니다 010 2965 2573 연락주세요
1,1,이니에스타,금카 적당한 알파 팝니다 쪽지주세요
2,2,이니에스타,** 삭제된 글입니다 **
3,3,이니에스타,금카삽니다!!!
4,4,이니에스타,카톡 keyaid90
...,...,...,...
183,183,다비드 은고그,5카 구매완료 판매자분 감사합니다
184,184,다비드 은고그,1카 삽니다 톡주세요wjdwngur421
185,185,다비드 은고그,1카 강장매물이랑 교환합니다 톡주세요 wjdwngur421
186,186,킹,금카 삽니다.
